Beat-n game with 15 pt limit.  Also, ties count towards "beat-n" count (assumed that ties can be broken via base dish selection.)

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JLD2

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

const MXV = 9
const MNV = 1

function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end

function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 15 and filter nondominated

In [4]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]>=0
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function play_random_vs_random(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_random(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end


lib0 = get_library(15);
lib=filter_nondominated2(lib0);
payoffs = get_payoffs(lib);
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));
libi = [spud2int(ff) for ff in lib];
nlib = length(lib)

1396

In [8]:
for ff in lib[[rand(1:length(lib)) for i in 1:100]]
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end


 131352 113631 316122 512232 121812 172311 212343 143241 415311 322431 117213 123711 243231 126411 172122 813111 342213 242322 622311 522114 223224 332115 112191 213216 123423 111228 121173 154311 213225 112272 263211 312333 119112 214521 322611 422124 212181 212451 312513 114117 415221 141423 116133 142422 415131 133143 326112 314115 317211 312342 112353 212163 216411 334221 225321 122217 234222 513321 116241 312117 131253 253311 315231 332223 114216 236112 323331 123432 213612 172122 213243 114513 113433 121155 151215 322512 214134 214224 142521 126312 313215 121533 122433 112236 117114 452112 413232 224124 222243 134133 622212 415311 117213 125115 225222 117141 235212 115215 223215 233421

In [194]:
a1 = int2ind[131352]
b1 = int2ind[219111]
#a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
#a_win, as, bs = play_random_vs_random(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
#a_win, as, bs = play_random_vs_random(payoffs, [a1],[b1])
a_win, as, bs = play_random_vs_random(payoffs)
display_game(payoffs, as, bs)

122712 {0}. 714111 {1}. 
317121 {1}. 133512 {1}. 
512322 {1}. 116124 {1}. 
121218 {1}. 523221 {2}. 
234231 {2}. 326211 {3}. 
151332 {3}. 224322 {4}. 
133431 {4}. 332142 {4}. 
133323 {6}. 222513 {6}. 
317112 {6}. 132513 {9}. 
417111 {9}. 


## MCTS

In [337]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+161) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-161 for i in 1:2:(2*length(s)-1)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

function propagate_win(tree::Dict{String, Vector{Int64}}, leaf::String, win::Int64)::Dict{String, Vector{Int64}}
    for i in 0:length(leaf)
        if i==0
            node=""
        else
            node=leaf[1:(2*i-1)]
        end
        if haskey(tree, node)
            tree[node][2] += 1
            tree[node][1] += win
        end
    end
    return tree
end

function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

function check_and_propagate_terminal(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    children = get_children(payoffs, leaf)
    if length(children) == 0
        # indicate terminal
        tree[leaf][3]=1
        if mod(length(leaf), 2)==0
            #p1 lost
            tree = propagate_win(tree, leaf, 0)
        else
            #p1 won
            tree = propagate_win(tree, leaf, 1)
        end
    end
    return tree
end

function try_add_node(tree::Dict{String, Vector{Int64}}, node::String)::Dict{String, Vector{Int64}}
    if haskey(tree, node)
        return tree
    else
        tree[node] = [0,0,0]
        return tree
    end
end

function query_node(tree, node)::Array{Int64}
    if haskey(tree, node)
        return tree[node]
    else
        return [0,0,0]
    end
end

function expand_leaf(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # check if leaf is terminal
    tree = check_and_propagate_terminal(payoffs, tree, leaf)
    if tree[leaf][3]==1
        return tree
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    end
    tree = try_add_node(tree, newleaf)
    tree = check_and_propagate_terminal(payoffs, tree, newleaf)
    # play the match
    as, bs = split_arrs(string_to_intarr(newleaf))
    a_win, as2, bs2 = play_random_vs_random(payoffs, as, bs)
    # propagate result
    if a_win == 1
        tree = propagate_win(tree, newleaf, 1)
    else
        tree = propagate_win(tree, newleaf, 0)
    end
    return tree
end

function sample_next_node(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, 
        explore_const::Float64=1.4)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if minimum(counts) == 0
        sel = children[rand(findall(counts .== 0))]
        return sel
    end
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i] + explore_const * sqrt((log(n)/counts[i]))
        end
    end
    sel = children[rand(findall(scores .== maximum(scores)))]
    return sel
end
                            
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String="")::String
    while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end
                                    
# exploit mode
function sample_next_node2(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, beta::Float64 = 1.0)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i]
        end
    end
    scores = [exp(beta*x) for x in scores]
    scores = scores./sum(scores)
    cs = cumsum(scores)
    z = rand(1)[1]
    sel = children[minimum(findall(cs .>= z))]
    return sel
end

function display_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    print("Node: ")
    println(string_to_intarr(node))
    as, bs = split_arrs(string_to_intarr(node))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(query_node(tree,node)[1])
    print("/")
    print(query_node(tree,node)[2])
    print("=")
    s = string(query_node(tree,node)[1]/query_node(tree,node)[2])
    println(s[1:min(5, length(s))])
    if query_node(tree,node)[3]==1
        println("[terminal] ")
    end
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            if haskey(tree, "")
                display_node(payoffs, tree, "")
                println()
            end
        else
            if haskey(tree, leaf[1:i])
                display_node(payoffs, tree, leaf[1:i])
                println()
            end
        end
    end
end

function display_choices(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    children = get_children(payoffs, node)
    display_node(payoffs, tree, node)
    for c in children
        println()
        display_node(payoffs, tree, c)
    end
end

function create_tree(node::String = "")::Dict{String, Vector{Int64}}
    return Dict(node => [0,0,0])
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String = "")::Dict{String, Vector{Int64}}
    node = random_leaf(payoffs, tree, node)
    tree = try_add_node(tree, node)
    tree = check_and_propagate_terminal(payoffs, tree, node)
    tree = expand_leaf(payoffs, tree, node)
    return tree
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}; 
        node::String = "", iters::Int64 = 1000)::Dict{String, Vector{Int64}}
    for i in 1:iters
        tree = grow_tree(payoffs, tree, node)
    end
    return tree
end

grow_tree (generic function with 2 methods)

In [338]:
# char_inds = vcat([i for i in 161:767], [i for i in 890:1156], [i for i in 1160:1423], [i for i in 2000:3000])
# for i in char_inds[500:end]
#     print(i)
#     print("u")
#     s = string("a", convert(Char, i), "b")
#     print(s[2])
#     s[4]
#     print(" ")
# end

In [339]:
for i in 161:(nlib + 161)
    s = string("a", convert(Char, i), "b")
    print(s[2])
end

¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĬĭĮįİıĲĳĴĵĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŉŊŋŌōŎŏŐőŒœŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſƀƁƂƃƄƅƆƇƈƉƊƋƌƍƎƏƐƑƒƓƔƕƖƗƘƙƚƛƜƝƞƟƠơƢƣƤƥƦƧƨƩƪƫƬƭƮƯưƱƲƳƴƵƶƷƸƹƺƻƼƽƾƿǀǁǂǃǄǅǆǇǈǉǊǋǌǍǎǏǐǑǒǓǔǕǖǗǘǙǚǛǜǝǞǟǠǡǢǣǤǥǦǧǨǩǪǫǬǭǮǯǰǱǲǳǴǵǶǷǸǹǺǻǼǽǾǿȀȁȂȃȄȅȆȇȈȉȊȋȌȍȎȏȐȑȒȓȔȕȖȗȘșȚțȜȝȞȟȠȡȢȣȤȥȦȧȨȩȪȫȬȭȮȯȰȱȲȳȴȵȶȷȸȹȺȻȼȽȾȿɀɁɂɃɄɅɆɇɈɉɊɋɌɍɎɏɐɑɒɓɔɕɖɗɘəɚɛɜɝɞɟɠɡɢɣɤɥɦɧɨɩɪɫɬɭɮɯɰɱɲɳɴɵɶɷɸɹɺɻɼɽɾɿʀʁʂʃʄʅʆʇʈʉʊʋʌʍʎʏʐʑʒʓʔʕʖʗʘʙʚʛʜʝʞʟʠʡʢʣʤʥʦʧʨʩʪʫʬʭʮʯʰʱʲʳʴʵʶʷʸʹʺʻʼʽʾʿˀˁ˂˃˄˅ˆˇˈˉˊˋˌˍˎˏːˑ˒˓˔˕˖˗˘˙˚˛˜˝˞˟ˠˡˢˣˤ˥˦˧˨˩˪˫ˬ˭ˮ˯˰˱˲˳˴˵˶˷˸˹˺˻˼˽˾˿̴̵̶̷̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿̀́͂̓̈́͆͊͋͌̕̚ͅ͏͓͔͕͖͙͚͐͑͒͗͛ͣͤͥͦͧͨͩͪͫͬͭͮͯ͘͜͟͢͝͞͠͡ͰͱͲͳʹ͵Ͷͷ͸͹ͺͻͼͽ;Ϳ΀΁΂΃΄΅Ά·ΈΉΊ΋Ό΍ΎΏΐΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡ΢ΣΤΥΦΧΨΩΪΫάέήίΰαβγδεζηθικλμνξοπρςστυφχψωϊϋόύώϏϐϑϒϓϔϕϖϗϘϙϚϛϜϝϞϟϠϡϢϣϤϥϦϧϨϩϪϫϬϭϮϯϰϱϲϳϴϵ϶ϷϸϹϺϻϼϽϾϿЀЁЂЃЄЅІЇЈЉЊЋЌЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяѐёђѓєѕіїјљњћќѝўџѠѡѢѣѤѥѦѧѨѩѪѫѬѭѮѯѰѱѲѳѴѵѶѷѸѹѺѻѼѽѾѿҀҁ҂҃҄҅҆҇҈

In [116]:
s = intarr_to_string([i for i in 1:nlib])

"¢£¤¥¦§¨©ª«¬\uad®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĬĭĮįİıĲĳĴĵĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŉŊŋŌōŎŏŐőŒœŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſƀƁƂƃƄƅƆƇƈƉƊƋƌƍƎƏƐƑƒƓƔƕƖƗƘƙƚƛƜƝƞƟƠơƢƣƤƥƦƧƨƩƪƫƬƭƮƯ" ⋯ 1712 bytes ⋯ "ԈԉԊԋԌԍԎԏԐԑԒԓԔԕԖԗԘԙԚԛԜԝԞԟԠԡԢԣԤԥԦԧԨԩԪԫԬԭԮԯ\u530ԱԲԳԴԵԶԷԸԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇՈՉՊՋՌՍՎՏՐՑՒՓՔՕՖ\u557\u558ՙ՚՛՜՝՞՟ՠաբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆևֈ։֊\u58b\u58c֍֎֏\u590ְֱֲֳִֵֶַָֹֺֻּֽ֑֖֛֢֣֤֥֦֧֪֚֭֮֒֓֔֕֗֘֙֜֝֞֟֠֡֨֩֫֬֯־ֿ׀ׁׂ׃ׅׄ׆ׇ\u5c8\u5c9\u5ca\u5cb\u5cc\u5cd\u5ce\u5cfאבגדהוזחטיךכלםמןנסעףפץצקרשת\u5eb\u5ec\u5ed\u5eeׯװױײ׳״\u5f5\u5f6\u5f7\u5f8\u5f9\u5fa\u5fb\u5fc\u5fd\u5fe\u5ff\u600\u601\u602\u603\u604\u605؆؇؈؉؊؋،؍؎؏ؐؑؒؓؔؕ"

In [117]:
is = string_to_intarr(s)

1396-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 1385
 1386
 1387
 1388
 1389
 1390
 1391
 1392
 1393
 1394
 1395
 1396

In [118]:
for i in 1:nlib
    s[2*i-1]
end

## Play vs MCTS
### CPU first

In [145]:
n_mc = 1
beta = 20.0
game = ""
tree = create_tree(game)
display_node(payoffs, create_tree(game), game)

Node: 
Stats: 0/0=NaN


In [146]:
# player turn
pmove = int2ind[316122]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: ҵ
316122 {0}. 
Stats: 0/0=NaN


In [154]:
tree = create_tree(game);

In [161]:
@time tree = grow_tree(payoffs, tree, node=game, iters=n_mc)

LoadError: InterruptException:

In [144]:
# MCTS turn
tree = create_tree(game);
@time tree = grow_tree(payoffs, tree, node=game, iters=n_mc);
game = sample_next_node2(payoffs, tree, game, beta);
#display_node(payoffs, create_tree(game), game)
display_node(payoffs, tree, game)

LoadError: InterruptException:

In [ ]:
# player turn
pmove = int2ind[316122]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, tree, game)

## Mate in X analysis

In [340]:
# mate in 1 checker
function mi1_checker(payoffs::Array{Int64}, leaf::String, verbose::Bool=false)::Int64
    ch = get_children(payoffs, node00)
    if length(ch)==0
        return -1
    else
        lens = [length(get_children(payoffs, c)) for c in ch]
        if minimum(lens)==0
            if verbose
                println("---------mate in 1-----------")
                display_node(payoffs, create_tree(leaf), leaf)
                println("::::play the following::::")
                chs = ch[findall(lens .== 0)]
                inds = [string_to_intarr(c)[end] for c in chs]
                println(libi[inds])
            end
            return 1
        end
    end
    return 0            
end

mi1_checker (generic function with 2 methods)

In [347]:
a_win, as, bs = play_random_vs_random(payoffs)
display_game(payoffs, as, bs)

node0 = intarr_to_string(merge_arrs(as, bs))
for i in 1:length(node0)
    node00 = node0[1:(2*i-1)]
    v = mi1_checker(payoffs, node00, true)
end

153213 {0}. 141531 {1}. 
512511 {1}. 133341 {2}. 
322521 {2}. 112731 {2}. 
113415 {2}. 214521 {4}. 
515112 {4}. 215232 {5}. 
131352 {5}. 516111 {5}. 
216321 {5}. 417111 {6}. 
417111 {6}. 417111 {7}. 
317112 {7}. 
---------mate in 1-----------
Node: [599, 508, 1295, 448, 1072, 94, 119, 750, 1312, 759, 391, 1315]
153213 {0}. 141531 {1}. 
512511 {1}. 133341 {2}. 
322521 {2}. 112731 {2}. 
113415 {2}. 214521 {4}. 
515112 {4}. 215232 {5}. 
131352 {5}. 516111 {5}. 
Stats: 0/0=NaN
::::play the following::::
[317112]
---------mate in 1-----------
Node: [599, 508, 1295, 448, 1072, 94, 119, 750, 1312, 759, 391, 1315, 775, 1218]
153213 {0}. 141531 {1}. 
512511 {1}. 133341 {2}. 
322521 {2}. 112731 {2}. 
113415 {2}. 214521 {4}. 
515112 {4}. 215232 {5}. 
131352 {5}. 516111 {5}. 
216321 {5}. 417111 {6}. 
Stats: 0/0=NaN
::::play the following::::
[151143, 217311, 317112]
---------mate in 1-----------
Node: [599, 508, 1295, 448, 1072, 94, 119, 750, 1312, 759, 391, 1315, 775, 1218, 1218, 1218]
153213 {0}

## Combo search

In [348]:
min(3,4)

3

In [372]:
combos = Array{Int64}(undef, (10000,3))
ord = sortperm([rand() for i in 1:nlib])
nc = 0
# 3-combos
for i in ord[1:(nlib-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(nlib-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            print(".")
            for k in ord[(j+1):nlib]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = libi[[i,j,k]]
                    println("")
                    println("----3-COMBO!-----")
                    println(libi[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:]

428 .............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
----3-COMBO!-----
[132441, 217122, 615111]
...........................................................................................................................................................................................................................................................................................................................

.............................................................1274 .........................................................................................................................730 .........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................1352 ...........................................1363 ................................809 .......................................................

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................941 ....................................................................................................................................

...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................288 .....................................................................................................................................................

.............................................................................................................................................................................................................................................................................................603 .......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

...

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................747 ........................................................................................................................................................................................................................................................................................

............................................................................................................................................................................................................................................................................................
----3-COMBO!-----
[116133, 141423, 615111]
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Excessive output truncated after 524288 bytes.

..

1013×3 Matrix{Int64}:
 132441  217122  615111
 122253  516111  151431
 122253  141531  416121
 415221  121524  151233
 112344  141432  416112
 112344  516111  151323
 112344  141423  416112
 121641  516111  151323
 152322  131154  516111
 152322  122145  516111
 152322  122514  516111
 152322  122154  516111
 152322  121164  516111
      ⋮          
 151323  131541  515121
 151323  121155  415113
 151323  121155  516111
 151323  121155  415212
 151323  121155  415122
 151323  121155  416112
 151323  121155  515211
 151323  121155  425112
 151323  121155  415311
 151323  121155  515112
 151323  121155  515121
 151323  222144  515112

In [373]:
combo_elts = sort(unique(vcat(combos[:, 1], combos[:, 2], combos[:, 3])));
elt_counts = [sum(combos .== e) for e in combo_elts]
combo_elts[sortperm(-elt_counts)[1:25]]

25-element Vector{Int64}:
 516111
 416112
 151323
 121155
 615111
 515112
 151431
 416121
 141432
 141423
 416211
 141234
 141333
 515121
 515211
 131514
 122424
 151233
 112515
 142323
 131541
 151422
 121254
 121524
 131442

In [374]:
elt_counts[sortperm(-elt_counts)[1:25]]

25-element Vector{Int64}:
 286
 250
 211
 101
  94
  84
  75
  71
  69
  66
  57
  52
  50
  46
  45
  43
  40
  40
  38
  37
  36
  36
  35
  35
  35

In [379]:
ss = Array{String}(undef, size(combos)[1])
for i in 1:size(combos)[1]
    a,b,c = sort(combos[i, :])
    ss[i] = string(c,",",b,",",a)
end

In [381]:
for s in sort(ss)
    print("")
    print(s)
    print("]")

1013-element Vector{String}:
 "316122,141441,112614"
 "415113,151323,121155"
 "415113,151323,121155"
 "415113,151323,121155"
 "415113,151323,121155"
 "415113,151431,131154"
 "415113,151431,131154"
 "415113,151431,131154"
 "415122,151323,121155"
 "415122,151323,121155"
 "415122,151323,121155"
 "415122,151323,121155"
 "415122,151323,121155"
 ⋮
 "615111,217131,132423"
 "615111,217212,131433"
 "615111,217221,131244"
 "615111,217221,131244"
 "615111,217221,131433"
 "615111,217221,131514"
 "615111,217221,131514"
 "615111,218112,131433"
 "615111,218121,131433"
 "615111,218121,131514"
 "615111,218121,131514"
 "615111,317211,131244"